In [1]:
import os
import argparse
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# pytorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, datasets

# sci-kit learn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# self-class
from model import UNet
#from dataset import *
from util import *

#Random seed
np.random.seed(7)
random.seed(7)
random_state=7

In [2]:
train = pd.read_csv('./datasets/train.csv')
test  = pd.read_csv('./datasets/test.csv')
submission = pd.read_csv('./datasets/submission.csv')

In [30]:
test

,id,letter,0,1,2,3,4,5,6,7,...,774,775,776,777,778,779,780,781,782,783
0,2049,L,0,4,0,2,4,2,3,1,...,2,0,4,2,2,4,3,4,1,4
1,2050,C,4,1,4,0,1,1,0,2,...,0,3,2,4,2,4,2,2,1,2
2,2051,S,0,4,0,1,3,2,3,0,...,1,3,2,0,3,2,3,0,1,4
3,2052,K,2,1,3,3,3,4,3,0,...,3,0,3,2,4,1,0,4,4,4
4,2053,W,1,0,1,1,2,2,1,4,...,4,3,1,4,0,2,1,2,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20475,22524,P,1,2,1,1,0,0,0,2,...,0,1,3,0,3,0,4,3,1,4
20476,22525,S,4,1,1,4,0,0,1,1,...,1,3,1,0,0,1,3,1,2,0
20477,22526,B,4,2,1,3,2,1,3,0,...,3,2,3,4,1,0,3,3,1,1
20478,22527,K,1,1,2,3,4,0,4,3,...,2,0,0,4,3,3,3,4,4,2


In [16]:
df_input = train.iloc[:,3:]
df_input

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
1614,3,0,0,4,4,4,4,3,0,0,...,3,0,4,3,1,0,3,3,1,4
1928,3,1,4,2,3,0,1,4,1,4,...,2,0,1,2,2,4,3,1,2,0
1343,4,1,4,3,2,1,1,0,2,3,...,3,0,0,2,4,4,1,0,2,0
1139,4,3,3,4,4,1,1,0,3,0,...,2,3,3,4,3,4,2,0,0,2
1616,2,3,1,4,4,1,2,4,1,1,...,4,1,1,1,3,0,0,4,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,2,0,2,0,1,4,1,0,0,2,...,1,3,2,2,1,2,1,1,2,3
502,4,1,3,1,3,2,4,1,2,4,...,2,3,2,2,1,2,2,2,1,2
537,2,1,0,2,2,2,1,1,1,3,...,0,4,2,0,0,1,3,0,3,3
1220,4,3,4,2,0,3,1,2,1,4,...,3,4,1,4,2,3,3,4,3,3


In [ ]:
data_dir = './datasets/train.csv'
data = pd.read_csv(data_dir)

del data['id']
del data['letter']

label = data['digit']
input = []

for i in range(2048):
    temp = data.iloc[i,1:].values.reshape(28, 28,1)
    input.append(temp)

# list 자료형으로는 img를 나타낼 수 없으므로 numpy 자료형으로 바꾸어 준다.    
# 데이터 타입 에러 발생으로 np.float32를 추가해준다
input = (np.array(input)).astype(np.float32)
label = (np.array(label)).astype(np.float32)

if input.ndim == 3:
    input = input[:, :, :, np.newaxis]

input = np.moveaxis(input[:, :, :, :], -1, 1)

#input.shape

In [76]:
data_dir = './datasets/train.csv'
data = pd.read_csv(data_dir)

x, y = train_test_split(data, test_size=0.125, random_state=7777)

len(x), len(y)

(1792, 256)

In [26]:
## define dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
        # __getitem__()에 넣기 위해 list label/input 생성
        label = self.data['digit']
        lst_label = list(label.values)
        
        df_input = self.data.iloc[:,3:]
        lst_input = []

        for i in range(len(df_input)):
            temp = df_input.iloc[i, :].values.reshape(28, 28) 
            lst_input.append(temp)

        self.lst_label = lst_label
        self.lst_input = lst_input
        
    def __len__(self):
        return len(self.lst_label)

    def __getitem__(self, index):
        # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 리턴
        label = self.lst_label[index]
        input = self.lst_input[index]
        
        # list 자료형으로는 img를 나타낼 수 없으므로 numpy 자료형으로 바꾸어 준다.
        # 데이터 타입 에러 발생으로 np.float32를 추가해준다. (하기 정규화시 필요)
        label = (np.array(label)).astype(np.float32)
        input = (np.array(input)).astype(np.float32)
        
        input = input / 255.0

        if input.ndim ==2:
            input = input[:, :, np.newaxis]
            
        data = {'input': input, 'label': label}

        if self.transform:
            data = self.transform(data)

        return data


## define transform

# nnConv2D 는 nSamples x nChannels x Height x Width 의 4차원 Tensor를 입력을 기본으로 하고
# 샘플 수에 대한 차원이 없을땐 채널, 세로, 가로만 입력해도 되는듯 하다.

class ToTensor(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        # ( 28, 28, 1) → (1, 28, 28)
        # (가로, 세로, 채널) → (채널, 가로, 세로)
        # 값을 확인해봐도 자리가 바뀌는 게 아니라 순서가 바뀌어 있다.
        input = np.moveaxis(input[:, :, :], -1, 0)

        data = {'label': torch.from_numpy(label), 'input': torch.from_numpy(input)}

        return data


class Normalization(object):
    def __init__(self, mean=0.5, std=0.5):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        label, input = data['label'], data['input']

        input = (input - self.mean) / self.std

        return data


class RandomFlip(object):
    def __call__(self, data):
        label, input = data['label'], data['input']

        if np.random.rand() > 0.5:
            label = np.fliplr(label)
            input = np.fliplr(input)

        if np.random.rand() > 0.5:
            label = np.fliqud(label)
            input = np.fliqud(input)

        data = {'label': label, 'input': input}

        return data

In [28]:
data_dir = './datasets'

transform = transforms.Compose([Normalization(mean=0.5, std=0.5), ToTensor()])

load_data = pd.read_csv(os.path.join(data_dir, 'train.csv'))
train, val = train_test_split(load_data, test_size=0.125, random_state=7)

dataset_train = Dataset(train, transform=transform)
loader_train = DataLoader(dataset_train, batch_size=8, shuffle=True, num_workers=4)

dataset_val = Dataset(val, transform=transform)
loader_val = DataLoader(dataset_val, batch_size=8, shuffle=False, num_workers=4)

for batch, data in enumerate(loader_train, 1):
    print (batch, data)

1 {'label': tensor([9., 5., 7., 5., 0., 4., 2., 3.]), 'input': tensor([[[[0.0039, 0.0157, 0.0000,  ..., 0.0039, 0.0118, 0.0078],
          [0.0078, 0.0039, 0.0000,  ..., 0.0118, 0.0039, 0.0078],
          [0.0078, 0.0118, 0.0078,  ..., 0.0039, 0.0157, 0.0000],
          ...,
          [0.0078, 0.0118, 0.0118,  ..., 0.0039, 0.0078, 0.0157],
          [0.0118, 0.0000, 0.0157,  ..., 0.0157, 0.0078, 0.0078],
          [0.0157, 0.0157, 0.0078,  ..., 0.0000, 0.0039, 0.0157]]],


        [[[0.0157, 0.0000, 0.0118,  ..., 0.0157, 0.0000, 0.0039],
          [0.0000, 0.0000, 0.0078,  ..., 0.0000, 0.0118, 0.0039],
          [0.0118, 0.0157, 0.0039,  ..., 0.0118, 0.0000, 0.0039],
          ...,
          [0.0039, 0.0118, 0.0157,  ..., 0.0118, 0.0118, 0.0000],
          [0.0078, 0.0000, 0.0078,  ..., 0.0118, 0.0039, 0.0078],
          [0.0039, 0.0000, 0.0118,  ..., 0.0078, 0.0000, 0.0118]]],


        [[[0.0039, 0.0000, 0.0157,  ..., 0.0118, 0.0000, 0.0157],
          [0.0000, 0.0078, 0.0118,  ..., 

79 {'label': tensor([1., 7., 7., 2., 2., 3., 6., 1.]), 'input': tensor([[[[0.0078, 0.0000, 0.0000,  ..., 0.0078, 0.0078, 0.0078],
          [0.0078, 0.0078, 0.0000,  ..., 0.0118, 0.0000, 0.0118],
          [0.0039, 0.0039, 0.0039,  ..., 0.0118, 0.0078, 0.0000],
          ...,
          [0.0000, 0.0039, 0.0118,  ..., 0.0078, 0.0000, 0.0118],
          [0.0039, 0.0118, 0.0000,  ..., 0.0157, 0.0000, 0.0039],
          [0.0118, 0.0118, 0.0157,  ..., 0.0078, 0.0039, 0.0157]]],


        [[[0.0078, 0.0157, 0.0078,  ..., 0.0039, 0.0000, 0.0039],
          [0.0157, 0.0039, 0.0157,  ..., 0.0039, 0.0118, 0.0000],
          [0.0157, 0.0039, 0.0157,  ..., 0.0000, 0.0118, 0.0157],
          ...,
          [0.0118, 0.0157, 0.0118,  ..., 0.0039, 0.0157, 0.0078],
          [0.0078, 0.0157, 0.0078,  ..., 0.0039, 0.0000, 0.0039],
          [0.0078, 0.0118, 0.0078,  ..., 0.0000, 0.0039, 0.0000]]],


        [[[0.0118, 0.0078, 0.0078,  ..., 0.0157, 0.0157, 0.0039],
          [0.0000, 0.0000, 0.0157,  ...,

159 {'label': tensor([8., 7., 6., 4., 9., 5., 2., 4.]), 'input': tensor([[[[0.0039, 0.0157, 0.0157,  ..., 0.0157, 0.0118, 0.0039],
          [0.0000, 0.0157, 0.0039,  ..., 0.0039, 0.0118, 0.0157],
          [0.0078, 0.0039, 0.0000,  ..., 0.0118, 0.0039, 0.0000],
          ...,
          [0.0078, 0.0078, 0.0000,  ..., 0.0078, 0.0118, 0.0078],
          [0.0157, 0.0157, 0.0000,  ..., 0.0000, 0.0118, 0.0000],
          [0.0118, 0.0000, 0.0039,  ..., 0.0000, 0.0039, 0.0157]]],


        [[[0.0000, 0.0157, 0.0000,  ..., 0.0039, 0.0000, 0.0157],
          [0.0078, 0.0039, 0.0000,  ..., 0.0157, 0.0157, 0.0078],
          [0.0039, 0.0039, 0.0039,  ..., 0.0000, 0.0118, 0.0157],
          ...,
          [0.0078, 0.0000, 0.0000,  ..., 0.0118, 0.0078, 0.0157],
          [0.0118, 0.0078, 0.0039,  ..., 0.0157, 0.0039, 0.0078],
          [0.0157, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0039]]],


        [[[0.0039, 0.0078, 0.0039,  ..., 0.0039, 0.0118, 0.0157],
          [0.0000, 0.0078, 0.0000,  ...

AttributeError: 'list' object has no attribute 'shape'

In [31]:
## import libraries

import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

## 네트워크 구축하기
# Image size = 28*28

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    # self.num_flat_features() 메서드는 input 텐서의 총 parameter 갯수이다.
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features



net=Net()

print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
